## Statistical analysis

Import libraries

In [ ]:
import pandas as pd
import numpy as np

Read database

In [ ]:
TAC_DATA = pd.read_parquet('TAC_db.parquet')

Escalated

In [ ]:
### Observe the categories
TAC_DATA['INCIDENT_URGENCY_NAME'].unique()

In [ ]:
### Fill NA with NONE
TAC_DATA['INCIDENT_URGENCY_NAME'] = TAC_DATA['INCIDENT_URGENCY_NAME'].fillna('NONE')

In [ ]:
### Group by three categories and count the number of TAC cases
DATA_URGENCY_SERVICE_C = TAC_DATA.groupby(['BRANCH_PARTY_SSOT_PARTY_ID_INT','INCIDENT_URGENCY_NAME','SERVICE_PROGRAM']).agg({'INCIDENT_NUMBER':'count'}).reset_index()

In [ ]:
DATA_URGENCY_SERVICE_C

In [ ]:
## agg sum 
PIVOT_TABLE_URGENCY = DATA_URGENCY_SERVICE_C.pivot_table(index='BRANCH_PARTY_SSOT_PARTY_ID_INT',columns='INCIDENT_URGENCY_NAME',values='INCIDENT_NUMBER',aggfunc='sum').fillna(0)

In [ ]:
## New variable with the addition of the categories
PIVOT_TABLE_URGENCY['TOTAL'] = PIVOT_TABLE_URGENCY['B2B Dormant']+PIVOT_TABLE_URGENCY['B2B Escalated']+PIVOT_TABLE_URGENCY['B2B Transferred To Partner']+PIVOT_TABLE_URGENCY['Escalated']+PIVOT_TABLE_URGENCY['NONE']

In [ ]:
## Calculate the percentage of the total
PIVOT_TABLE_URGENCY['PERESCALATED'] = PIVOT_TABLE_URGENCY['Escalated']+PIVOT_TABLE_URGENCY['B2B Escalated']/PIVOT_TABLE_URGENCY['TOTAL']

In [ ]:
## Statistics
PIVOT_TABLE_URGENCY.describe()

In [ ]:
## Check quantity 
(PIVOT_TABLE_URGENCY['PERESCALATED']>0).value_counts()

In [ ]:
## Relate with other column
TAC_DATA[TAC_DATA['INCIDENT_URGENCY_NAME'].isin(['B2B Escalated','Escalated'])]['MAX_SEVERITY_INT'].value_counts()

Outage

TAC_DATA['OUTAGE_FLAG'].value_counts()

In [ ]:
## Organize categories
TAC_DATA.loc[TAC_DATA['OUTAGE_FLAG']=='true','OUTAGE_FLAG']='Yes'
TAC_DATA.loc[TAC_DATA['OUTAGE_FLAG']=='false','OUTAGE_FLAG']='No'
TAC_DATA.loc[TAC_DATA['OUTAGE_FLAG']=='XYZ','OUTAGE_FLAG']='No'

In [ ]:
## Fill NAs with No
TAC_DATA['OUTAGE_FLAG'] = TAC_DATA['OUTAGE_FLAG'].fillna('No')

In [ ]:
## Group by party ID
DATA_OUTAGE_C = TAC_DATA.groupby(['BRANCH_PARTY_SSOT_PARTY_ID_INT','OUTAGE_FLAG','SERVICE_PROGRAM']).agg({'INCIDENT_NUMBER':'count'}).reset_index()

In [ ]:
DATA_OUTAGE_C

In [ ]:
## Create a pivot table
PIVOT_TABLE_OUTAGE = DATA_OUTAGE_C.pivot_table(index='BRANCH_PARTY_SSOT_PARTY_ID_INT',columns='OUTAGE_FLAG',values='INCIDENT_NUMBER',aggfunc='sum').fillna(0)

In [ ]:
PIVOT_TABLE_OUTAGE

In [ ]:
## Add another column
PIVOT_TABLE_OUTAGE['PERYES'] = PIVOT_TABLE_OUTAGE['Yes'] / (PIVOT_TABLE_OUTAGE['No']+PIVOT_TABLE_OUTAGE['Yes'])

In [ ]:
## Statistics
PIVOT_TABLE_OUTAGE.describe()

In [ ]:
## Check the condition
(PIVOT_TABLE_OUTAGE['PERYES']>0).value_counts()

In [ ]:
## Relate with other column
TAC_DATA[TAC_DATA['OUTAGE_FLAG'].isin(['Yes'])]['MAX_SEVERITY_INT'].value_counts()

Complexity

In [ ]:
TAC_DATA['COMPLEXITY_DESCR'].value_counts()

In [ ]:
## Create Pivot Table
COMPLEXITY_PIVOT = TAC_DATA.pivot_table(index='BRANCH_PARTY_SSOT_PARTY_ID_INT',columns='COMPLEXITY_DESCR',values='INCIDENT_NUMBER',aggfunc='count').fillna(0)

In [ ]:
## Add calculated columns
COMPLEXITY_PIVOT['Total'] = COMPLEXITY_PIVOT['0 Level -Procedural']+COMPLEXITY_PIVOT['1 Level -Basic']+COMPLEXITY_PIVOT['2 Level -Advanced']+COMPLEXITY_PIVOT['3 Level -Exceptionally Complex']
COMPLEXITY_PIVOT['PERCOMPLEX'] = COMPLEXITY_PIVOT['3 Level -Exceptionally Complex']/(COMPLEXITY_PIVOT['Total']) 

In [ ]:
## Statistics
COMPLEXITY_PIVOT.describe()

Entry Channel Name

In [ ]:
TAC_DATA['ENTRY_CHANNEL_NAME'].value_counts()

In [ ]:
b = TAC_DATA.groupby(['ENTRY_CHANNEL_NAME','INITIAL_SEVERITY_INT']).agg({'INCIDENT_NUMBER':'count'}).reset_index().pivot_table(index='ENTRY_CHANNEL_NAME',columns='INITIAL_SEVERITY_INT',values='INCIDENT_NUMBER').reindex().fillna(0)
b['TOTAL'] = b[1]+b[2]+b[3]+b[4]
b[1] = (b[1]/b['TOTAL'])*100
b[2] = (b[2]/b['TOTAL'])*100
b[3] = (b[3]/b['TOTAL'])*100
b[4] = (b[4]/b['TOTAL'])*100
b

In [ ]:
## Download
b.to_csv('Entry_Chanel_Severity.csv')

Min Max Severity

In [ ]:
## Create a calculated column to determine a change in severity
TAC_DATA['SEVERITY_INCREASED'] = 0
TAC_DATA.loc[(TAC_DATA['MIN_SEVERITY_INT']>TAC_DATA['MAX_SEVERITY_INT']),'SEVERITY_INCREASED'] = 1

In [ ]:
# By case
print((TAC_DATA['MIN_SEVERITY_INT']>TAC_DATA['MAX_SEVERITY_INT']).value_counts())

In [ ]:
# By account
print(TAC_DATA.groupby('BRANCH_PARTY_SSOT_PARTY_ID_INT').agg({'SEVERITY_INCREASED':'max'}).value_counts())

In [ ]:
## Create a pivot table
a = TAC_DATA.groupby(['SEVERITY_INCREASED','MAX_SEVERITY_INT']).agg({'INCIDENT_NUMBER':'count'}).reset_index().pivot_table(index='SEVERITY_INCREASED',columns='MAX_SEVERITY_INT',values='INCIDENT_NUMBER').reindex().fillna(0)
a['TOTAL'] = a[1]+a[2]+a[3]+a[4]
a[1] = (a[1]/a['TOTAL'])*100
a[2] = (a[2]/a['TOTAL'])*100
a[3] = (a[3]/a['TOTAL'])*100
a[4] = (a[4]/a['TOTAL'])*100
a

In [ ]:
## Download
a.to_csv('Severity_Increased.csv')

Problem code and Resolution code

In [ ]:
TAC_DATA.groupby(['PROBLEM_CODE','RESOLUTION_CODE_F']).agg({'INCIDENT_NUMBER':'count'}).reset_index().pivot_table(index='PROBLEM_CODE',columns='RESOLUTION_CODE_F',values='INCIDENT_NUMBER').reset_index().fillna(0)

In [ ]:
TAC_DATA[~TAC_DATA['TROUBLESHOOTING_DESCRIPTION'].isna()]['RESOLUTION_CODE_F'].value_counts()

In [ ]:
TAC_DATA[TAC_DATA['TROUBLESHOOTING_DESCRIPTION'].isna()]['RESOLUTION_CODE_F'].value_counts()

Resolution code and Request Type

In [ ]:
c = TAC_DATA.groupby(['REQUEST_TYPE_NAME','RESOLUTION_CODE_F']).agg({'INCIDENT_NUMBER':'count'}).reset_index().pivot_table(index='REQUEST_TYPE_NAME',columns='RESOLUTION_CODE_F',values='INCIDENT_NUMBER').reset_index().fillna(0)
c['TOTAL'] = c['3RD PARTY']+c['EDUCATION']+c['HW-RMA']+c['OTHERS']+c['SW-APP-LIC']
c['3RD PARTY'] = (c['3RD PARTY']/c'TOTAL'])*100
c['EDUCATION']= (c['EDUCATION']/c['TOTAL'])*100
c['HW-RMA'] = (c['HW-RMA']/c['TOTAL'])*100
c['OTHERS'] = (c['OTHERS']/c['TOTAL'])*100
c['SW-APP-LIC']= (c['SW-APP-LIC']/c['TOTAL'])*100
c

In [ ]:
## Download
c.to_csv('Request_type_Resolution_code.csv')

Reopen date

In [ ]:
## Create a new column
TAC_DATA['REOPEN'] = 0
TAC_DATA.loc[~TAC_DATA['LAST_REOPEN_DATE'].isna(),'REOPEN'] = 1

In [ ]:
a = TAC_DATA.groupby(['REOPEN','MAX_SEVERITY_INT']).agg({'INCIDENT_NUMBER':'count'}).pivot_table(index = 'REOPEN',columns='MAX_SEVERITY_INT',values='INCIDENT_NUMBER')
a['TOTAL'] = a[1]+a[2]+a[3]+a[4]
a[1] = (a[1]/a['TOTAL'])*100
a[2] = (a[2]/a['TOTAL'])*100
a[3] = (a[3]/a['TOTAL'])*100
a[4] = (a[4]/a['TOTAL'])*100
a

In [ ]:
a.to_csv('Reopen_Severity.csv')

Graphics

In [ ]:
import matplotlib
import plotly.graph_objects as go

Reopen Cases and Not Reopen cases

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x = TAC_DATA['MAX_SEVERITY_INT'][TAC_DATA['REOPEN']==0], histnorm='percent', name='No', marker_color="#00bceb" ) )
fig.add_trace(go.Histogram(x = TAC_DATA['MAX_SEVERITY_INT'][TAC_DATA['REOPEN']==1], histnorm='percent', name='Yes', marker_color="#0d274d") )
fig.update_xaxes(categoryorder='array', categoryarray= ['1', '2', '3', '4'])
fig.update_layout(
    title={'text': "Reopen Cases", 'xanchor':'center', 'x':0.5},
    xaxis_title="Max Severity",
    yaxis_title="Percent (%)",
    legend_title="Reopen",
    paper_bgcolor = 'rgba(255,255,255,1)',
    plot_bgcolor = 'rgba(255,255,255,1)',
)
fig.update_yaxes(gridcolor = 'rgba(200,200,200,1)')
fig.show()
# px.histogram(df,x = 'MAX_SEVERITY_INT',color = 'REOPEN',category_orders={'MAX_SEVERITY_INT':['1','2','3','4']},histnorm = 'percent',barmode = 'group',labels={0:'no', 1:'yes'})

In [ ]:
TAC_DATA[TAC_DATA['REOPEN']==0]['MAX_SEVERITY_INT'].value_counts().to_dict()

In [ ]:
list(a.keys())

Not Reopen Cases (Pie)

In [ ]:
a = TAC_DATA[TAC_DATA['REOPEN']==0]['MAX_SEVERITY_INT'].value_counts().to_dict()
fig = go.Figure()
fig.add_trace(go.Pie(labels=['1','2','3','4'],values=list(map(a.get,['1','2','3','4'])),sort =False, marker_colors = ['#0d274d', '#1e4471', '#00bceb', '#ced4da']) ) 
fig.update_layout(
    title={'text': "Not Reopen Cases", 'xanchor':'center', 'x':0.5},
    xaxis_title="Max Severity",
    yaxis_title="Percent (%)",
    legend_title="Severity",
    paper_bgcolor = 'rgba(255,255,255,1)',
    plot_bgcolor = 'rgba(255,255,255,1)',
)
fig.show()

Reopen cases

In [ ]:
a = TAC_DATA[TAC_DATA['REOPEN']==1]['MAX_SEVERITY_INT'].value_counts().to_dict()
fig = go.Figure()
fig.add_trace(go.Pie(labels=['1','2','3','4'],values=list(map(a.get,['1','2','3','4'])),sort =False, marker_colors = ['#0d274d', '#1e4471', '#00bceb', '#ced4da']) ) 
fig.update_layout(
    title={'text': "Reopen Cases", 'xanchor':'center', 'x':0.5},
    xaxis_title="Max Severity",
    yaxis_title="Percent (%)",
    legend_title="Severity",
    paper_bgcolor = 'rgba(255,255,255,1)',
    plot_bgcolor = 'rgba(255,255,255,1)',
)
fig.show()

Customer Activity Severity

In [ ]:
d = TAC_DATA.groupby(['CUSTOMER_ACTIVITY_CODE','MAX_SEVERITY_INT']).agg({'INCIDENT_NUMBER':'count'}).reset_index().pivot_table(index='CUSTOMER_ACTIVITY_CODE',columns='MAX_SEVERITY_INT',values='INCIDENT_NUMBER').reset_index()
d['TOTAL'] = d[1]+d[2]+d[3]+d[4]
d[1] = (d[1]/d['TOTAL'])*100
d[2] = (d[2]/d['TOTAL'])*100
d[3] = (d[3]/d['TOTAL'])*100
d[4] = (d[4]/d['TOTAL'])*100
d

In [ ]:
d.to_csv('Customer_activity_Severity.csv')

Complexity Severity

In [ ]:
e = TAC_DATA.groupby(['COMPLEXITY_DESCR','MAX_SEVERITY_INT']).agg({'INCIDENT_NUMBER':'count'}).reset_index().pivot_table(index='COMPLEXITY_DESCR',columns='MAX_SEVERITY_INT',values='INCIDENT_NUMBER').reset_index()
e['TOTAL'] = e[1]+e[2]+e[3]+e[4]
e[1] = (e[1]/e['TOTAL'])*100
e[2] = (e[2]/e['TOTAL'])*100
e[3] = (e[3]/e['TOTAL'])*100
e[4] = (e[4]/e['TOTAL'])*100
e

In [ ]:
e.to_csv('Complexity_Severity.csv')

WordClouds

In [ ]:
import plotly.express as px
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt

Sumary TXT

In [ ]:
txt = str(TAC_DATA['SR_SUMMARY_TXT'])
txt

In [ ]:
print(STOPWORDS)

In [ ]:
wc = WordCloud( background_color="white",stopwords=['been', 'off', 'few', 'same', 'against', 'we', 'of',
                                                     'only', 'by', 'he', 'all', 'r', 'not', "that's", 'they',
                                                      'itself', 'those', 'get', 'more', 'were', 'each', 'do', 'most', 'a', 'but',
                                                     'however', "you'll", "there's", 'him', 'herself', 'had', "can't", 'i', "let's", 'again',
                                                      'this', 'through', 'why', 'k', 'his', 'it', 'into', "what's", "didn't", "shan't", "i've", 'up',
                                                       'www', 'from', "hasn't", "couldn't", 'no', 'the', 'as', 'therefore', 'hence', 'until', "when's",
                                                        'shall', 'at', 'or', 'our', 'like', 'further', 'about', 'too', 'with', 'during', "she'd", 'some', 
                                                        "they've", 'has', 'over', 'an', "doesn't", 'here', 'you', "i'll", 'for', 'my', 'so', 'than', 'in',
                                                         'ours', "we'll", "he'll", 'nor', 'who', "i'd", 'when', "haven't", "who's", "hadn't", 'to', 'com', 'ever',
                                                          "she's", "how's", 'if', 'ought', 'their', "you're", 'down', "mustn't", 'myself', 'any', 'then', 'between',
                                                           'would', 'should', 'having', "he'd", 'on', "they'll", 'out', 'doing', "here's", "they're", 'yours', "i'm",
                                                            'these', 'have', 'themselves', 'such', 'http', 'while', 'hers', 'can', 'also', 'does', 'else', 'being',
                                                         "it's", 'yourself', "we'd", 'be', 'could', "they'd", 'under', "we've", "you've", 'there', 'was', "wasn't", 
                                                         'below', 'are', 'himself', 'above', 'me', 'which', 'that', 'them', 'what', "wouldn't", "isn't",
                                                           'theirs', "why's", 'and', "we're", "don't", 'is', 'after', 'its', 'other', "weren't", 'because',
                                                            'whom', 'otherwise', 'once', 'ourselves', "won't", 'own', 'cannot', 'just', "she'll", 'both', 
                                                            'yourselves', 'very', "you'd", 'your',"he's", 'her', 'before', "where's", 'how', 'she', 'did', 'am', "shouldn't", 'since', "aren't", 'where',
                                                            'respo','lentgth','due','name','w','personal','show','dpd','dtype','Po','Payr','Invites','SR_SUMMARY_TXT'],max_words=300,width=2000,height=1000
).generate(txt)
plt.axis('Off')
plt.imshow(wc)

Troubleshooting Description

In [ ]:
T= TAC_DATA['TROUBLESHOOTING_DESCRIPTION'].unique().tolist()
T

In [ ]:
T = str(T)
wc = WordCloud( background_color="white", stopwords=['been', 'off', 'few', 'same', 'against', 'we', 'of',
                                                     'only', 'by', 'he', 'all', 'r', 'not', "that's", 'they',
                                                      'itself', 'those', 'get', 'more', 'were', 'each', 'do', 'most', 'a', 'but',
                                                     'however', "you'll", "there's", 'him', 'herself', 'had', "can't", 'i', "let's", 'again',
                                                      'this', 'through', 'why', 'k', 'his', 'it', 'into', "what's", "didn't", "shan't", "i've", 'up',
                                                       'www', 'from', "hasn't", "couldn't", 'no', 'the', 'as', 'therefore', 'hence', 'until', "when's",
                                                        'shall', 'at', 'or', 'our', 'like', 'further', 'about', 'too', 'with', 'during', "she'd", 'some', 
                                                        "they've", 'has', 'over', 'an', "doesn't", 'here', 'you', "i'll", 'for', 'my', 'so', 'than', 'in',
                                                         'ours', "we'll", "he'll", 'nor', 'who', "i'd", 'when', "haven't", "who's", "hadn't", 'to', 'com', 'ever',
                                                          "she's", "how's", 'if', 'ought', 'their', "you're", 'down', "mustn't", 'myself', 'any', 'then', 'between',
                                                           'would', 'should', 'having', "he'd", 'on', "they'll", 'out', 'doing', "here's", "they're", 'yours', "i'm",
                                                            'these', 'have', 'themselves', 'such', 'http', 'while', 'hers', 'can', 'also', 'does', 'else', 'being',
                                                         "it's", 'yourself', "we'd", 'be', 'could', "they'd", 'under', "we've", "you've", 'there', 'was', "wasn't", 
                                                         'below', 'are', 'himself', 'above', 'me', 'which', 'that', 'them', 'what', "wouldn't", "isn't",
                                                           'theirs', "why's", 'and', "we're", "don't", 'is', 'after', 'its', 'other', "weren't", 'because',
                                                            'whom', 'otherwise', 'once', 'ourselves', "won't", 'own', 'cannot', 'just', "she'll", 'both', 
                                                            'yourselves', 'very', "you'd", 'your',"he's", 'her', 'before', "where's", 'how', 'she', 'did', 'am', "shouldn't", 'since', "aren't", 'where',
                                                            'respo','lentgth','due','name','w','personal','show','dpd','dtype','Po','Payr','Invites','None','TROUBLESHOOTING_DESCRIPTION','Length','Object','xa0','n','na','ma','dBm','error count']
                                                            ,max_words=100,width=2000,height=1000).generate(T)
plt.axis('Off')

plt.imshow(wc)